### @logger

In [2]:
def logger(function):
    def wrapper(*args, **kwargs):
        print(f"----- {function.__name__}: start -----")
        output = function(*args, **kwargs)
        print(f"----- {function.__name__}: end -----")
        return output

    return wrapper


@logger
def say_hello():
    print("Hello World!")


say_hello()

----- say_hello: start -----
Hello World!
----- say_hello: end -----


### @lru_cache
@lru_cache是Python内置装饰器，可以通过from functools import lru_cache引入。@lru_cache的作用是缓存函数的返回值，当缓存装满时，使用least-recently-used（LRU）算法丢弃最少使用的值。

@lru_cache装饰器适合用于输入输出不变且运行时间较长的任务，例如查询数据库、请求静态页面或一些繁重的处理。

In [6]:
from functools import lru_cache


@lru_cache(maxsize=128, typed=True)
def add(x, y):
    print(f"Calculating: {x} + {y}")
    return x + y


print(add(1, 2))  # 输出：Calculating: 1 + 2 \n 3
print(add(1, 2))  # 输出：3
print(add(1.0, 2.0))  # 输出：Calculating: 1.0 + 2.0 \n 3.0
print(add(1.0, 2.0))  # 输出：3.0


Calculating: 1 + 2
3
3
Calculating: 1.0 + 2.0
3.0
3.0


### @repeat
该装饰器的所用是多次调用被修饰函数。这对于调试、压力测试或自动化多个重复任务非常有用。

In [10]:
def repeat(number_of_times):
    def decorate(func):
        def wrapper(*args, **kwargs):
            for _ in range(number_of_times):
                func(*args, **kwargs)

        return wrapper

    return decorate


@repeat(5)
def hello_world():
    print("hello world")


hello_world()

hello world
hello world
hello world
hello world
hello world


### @timeit
该装饰器用来测量函数的执行时间并打印出来。这对调试和监控非常有用。

In [11]:
import time
from functools import wraps


def timeit(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        start = time.perf_counter()
        result = func(*args, **kwargs)
        end = time.perf_counter()
        print(f'{func.__name__} took {end - start:.6f} seconds to complete')
        return result

    return wrapper


@timeit
def process_data():
    time.sleep(1)


process_data()
# process_data took 1.000012 seconds to complete


process_data took 1.000658 seconds to complete


### @retry
当函数遇到异常时，该装饰器会强制函数重试多次。它接受三个参数：重试次数、捕获的异常以及重试之间的间隔时间。

其工作原理如下：
- wrapper函数启动num_retrys次迭代的for循环。
- 将被修饰函数放到try/except块中。每次迭代如果调用成功，则中断循环并返回结果。否则，休眠sleep_time秒后继续下一次迭代。
- 当for循环结束后函数调用依然不成功，则抛出异常。

In [13]:
import random
import time
from functools import wraps


def retry(num_retries, exception_to_check, sleep_time=0):
    """
    遇到异常尝试重新执行装饰器
    """

    def decorate(func):
        @wraps(func)
        def wrapper(*args, **kwargs):
            for i in range(1, num_retries + 1):
                try:
                    return func(*args, **kwargs)
                except exception_to_check as e:
                    print(f"{func.__name__} raised {e.__class__.__name__}. Retrying...")
                    if i < num_retries:
                        time.sleep(sleep_time)
            # 尝试多次后仍不成功则抛出异常
            raise e

        return wrapper

    return decorate


@retry(num_retries=3, exception_to_check=ValueError, sleep_time=1)
def random_value():
    value = random.randint(1, 5)
    if value == 3:
        raise ValueError("Value cannot be 3")
    return value


random_value()
# random_value raised ValueError. Retrying...
# 1

random_value()
# 5


5

### @count_call
@count_call 用于统计被修饰函数的调用次数。这里的调用次数会缓存在wraps的count属性中。

In [16]:
from functools import wraps


def count_call(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        wrapper.count += 1
        result = func(*args, **kwargs)
        print(f'{func.__name__} has been called {wrapper.count} times')
        return result

    wrapper.count = 0
    return wrapper


@count_call
def process_data():
    pass


process_data()
process_data()

process_data has been called 1 times
process_data has been called 2 times


### @rate_limited
@rate_limited装饰器会在被修饰函数调用太频繁时，休眠一段时间，从而限制函数的调用速度。

这在模拟、爬虫、接口调用防过载等场景下非常有用。

In [ ]:
import time
from functools import wraps


def rate_limited(max_per_second):
    min_interval = 1.0 / float(max_per_second)

    def decorate(func):
        last_time_called = [0.0]

        @wraps(func)
        def rate_limited_function(*args, **kargs):
            elapsed = time.perf_counter() - last_time_called[0]
            left_to_wait = min_interval - elapsed
            if left_to_wait > 0:
                time.sleep(left_to_wait)
            ret = func(*args, **kargs)
            last_time_called[0] = time.perf_counter()
            return ret

        return rate_limited_function

    return decorate



### @dataclass
Python 3.7 引入了@dataclass装饰器，将其加入到标准库，用于装饰类。它主要用于存储数据的类自动生成诸如`__init__`， `__repr__`， `__eq__`， `__lt__`，`__str__` 等特殊函数。这样可以减少模板代码，并使类更加可读和可维护。

In [17]:
from dataclasses import dataclass, asdict


@dataclass
class Person:
    first_name: str
    last_name: str
    age: int
    job: str

    def __eq__(self, other):
        if isinstance(other, Person):
            return self.age == other.age
        return NotImplemented

    def __lt__(self, other):
        if isinstance(other, Person):
            return self.age < other.age
        return NotImplemented


john = Person(first_name="John",
              last_name="Doe",
              age=30,
              job="doctor", )

anne = Person(first_name="Anne",
              last_name="Smith",
              age=40,
              job="software engineer", )

print(john == anne)
# False

print(anne > john)
# True

asdict(anne)
#{'first_name': 'Anne',
# 'last_name': 'Smith',
# 'age': 40,
# 'job': 'software engineer'}


False
True


{'first_name': 'Anne',
 'last_name': 'Smith',
 'age': 40,
 'job': 'software engineer'}

### @property
@property装饰器用于定义类属性，这些属性本质上是类实例属性的getter、setter和deleter方法。

通过使用@property装饰器，可以将方法定义为类属性，并将其作为类属性进行访问，而无需显式调用该方法。

如果您想在获取或设置值时添加一些约束和验证逻辑，使用@property装饰器会非常方便。

In [25]:
class Movie:
    def __init__(self, r):
        self._rating = r

    @property
    def rating(self):
        return self._rating

    @rating.setter
    def rating(self, r):
        if 0 <= r <= 5:
            self._rating = r
        else:
            raise ValueError("The movie rating must be between 0 and 5!")


batman = Movie(2.5)
print(batman.rating)
# 2.5

batman.rating = 4
print(batman.rating)
# 4

batman.rating = 10

# ---------------------------------------------------------------------------
# ValueError                                Traceback (most recent call last)
# Input In [16], in <cell line: 1>()
# ----> 1 batman.rating = 10
# Input In [11], in Movie.rating(self, r)
#      12     self._rating = r
#      13 else:
# ---> 14     raise ValueError("The movie rating must be between 0 and 5!")
#
# ValueError: The movie rating must be between 0 and 5!


2.5
4


ValueError: The movie rating must be between 0 and 5!